# Topic Modeling

Outline:
- 1) LDA (full sample)
- 2) BERTopic 

To be further developed... 

## 1) LDA (full sample)

In [ ]:
import pandas as pd

In [ ]:
# Repeat topic modeling for all publishers now (append all the samples)
df_all = pd.concat([pd.read_csv(f"../data/processed/newspapers/sample_{re.sub(r'\\W+','_ ', pub.lower()).strip('_')}.csv") for pub in publishers['publication']], ignore_index=True)

# Tokenization and stopword removal using regex and sklearn stopwords
custom_stopwords = ENGLISH_STOP_WORDS.union({'said', 'mr', 'also'})

def tokenize_and_clean(text):
    # Keep words with 3 or more alphabetic characters
    tokens = re.findall(r'\b[a-z]{3,}\b', str(text).lower())
    return [t for t in tokens if t not in custom_stopwords]

df_all['tokens'] = df_all['article'].apply(tokenize_and_clean)

# Create dictionary and corpus
dictionary = corpora.Dictionary(df_all['tokens'])
dictionary.filter_extremes(no_below=10, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in df_all['tokens']]

# Train LDA model
num_topics = 10
lda_model = models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15, random_state=42)

# Plot wordclouds
fig, axes = plt.subplots(2, 5, figsize=(20, 10), constrained_layout=True)
axes = axes.flatten()

for idx, ax in enumerate(axes):
    topic_words = dict(lda_model.show_topic(idx, 50))
    wc = WordCloud(width=500, height=300, background_color='white', max_words=50)
    wc.generate_from_frequencies(topic_words)
    ax.imshow(wc, interpolation='bilinear')
    ax.set_title(f'Topic {idx + 1}', fontsize=14)
    ax.axis('off')

plt.suptitle('LDA Topics – Word Clouds', fontsize=18)
plt.show()

In [ ]:
# Save LDA model and dictionary
lda_model.save("../models/topic_model/lda_model.gensim")
dictionary.save("../models/topic_model/lda_dictionary.dict")

In [ ]:
from gensim import corpora, models

# Load model and dictionary
lda_model = models.LdaModel.load("../models/topic_model/lda_model.gensim")
dictionary = corpora.Dictionary.load("../models/topic_model/lda_dictionary.dict")

In [ ]:
from gensim.models import CoherenceModel

coherence_model = CoherenceModel(model=lda_model, texts=df_all['tokens'], dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print(f'Coherence Score (c_v): {coherence_score:.4f}')
# A score above 0.4 is generally considered good for topic coherence.

In [ ]:
from datetime import datetime
import re

# --- Assumes these exist ---
# - df['tokens'] = list of preprocessed word tokens
# - df['date'] = parsed datetime
# - df['publication'] = publisher name
# - lda_model = trained gensim LdaModel
# - dictionary = gensim Dictionary used to train the model

# Load all samples from new processed newspapers folder
df_all = pd.concat([pd.read_csv(f"../data/processed/newspapers/sample_{re.sub(r'\\W+','_ ', pub.lower()).strip('_')}.csv") for pub in publishers['publication']], ignore_index=True)

# Step 1: Convert tokens to bag-of-words
corpus = [dictionary.doc2bow(text) for text in df_all['tokens']]

# Step 2: Get topic distribution for each article
def get_topic_dist(bow):
    # Return full-length vector with zero entries where necessary
    dist = lda_model.get_document_topics(bow, minimum_probability=0)
    return [prob for _, prob in dist]

df_all['topic_distribution'] = [get_topic_dist(doc) for doc in corpus]

# Step 3: Unpack topic distributions into separate columns
num_topics = lda_model.num_topics
topic_cols = [f'topic_{i}' for i in range(num_topics)]
df_topics = pd.DataFrame(df_all['topic_distribution'].tolist(), columns=topic_cols)

# Step 4: Combine with metadata
df_meta = df_all[['date', 'publication']].copy()
df_combined = pd.concat([df_meta, df_topics], axis=1)
df_combined['month'] = pd.to_datetime(df_combined['date'], format='mixed', errors='coerce').dt.to_period('M')

# Step 5: Aggregate topic shares by month and publisher
df_monthly_pub = df_combined.groupby(['month', 'publication'])[topic_cols].mean().reset_index()

# Step 6: Save to CSV
df_monthly_pub.to_csv('../data/processed/monthly_topic_shares_by_publisher.csv', index=False)

print("✅ Saved: 'monthly_topic_shares_by_publisher.csv'")

## 2) BERTopic

Goal: Topic shares by month-publisher using BERTopic for the topic modeling. 